In [ ]:
import cv2
import glob
import torch


In [ ]:
img_array = []
for filename in glob.glob('train/train_sharp_bicubic/X4/000/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 5, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [ ]:
import cv2

cam = cv2.VideoCapture("video.mp4")

currentframe = 0

while (True):
    # reading from frame
    ret, frame = cam.read()

    if ret:
        name = './data/frame' + str(currentframe) + '.jpg'
        print('Creating...' + name)

        cv2.imwrite(name, frame)

        currentframe += 1
    else:
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
import torch
from torch_enhance.models import SRResNet
from torch_enhance.datasets import Set14, Set5
from poutyne.framework import Model

pytorch_network = SRResNet(scale_factor=2, channels=3)

video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter('video2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 15, size)

frame_tensor = torch.from_numpy(img_array[0]).permute(2, 0, 1)
sr_img = model(frame_tensor)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
# 
#     frame_tensor = torch.from_numpy(frame).permute(2, 0, 1)
# 
#     sr_frame = model(frame_tensor)
# 
#     sr_frame = sr_frame.permute(1, 2, 0).numpy()
# 
#     out.write(sr_frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch_enhance.models import SRResNet

In [2]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.inputs = []
        self.targets = []
        self.transform = transform
        index = 0
        for folder in os.listdir('train/train_blur_bicubic/X4/'):
            index = index + 1
            if index >= 5:
                break
            for filename in os.listdir('train/train_blur_bicubic/X4/' + folder):
                img = cv2.imread('train/train_blur_bicubic/X4/' + folder + '/' + filename)
                img = img.transpose((2, 0, 1)).astype(np.float32)
                self.inputs.append(img)
        index = 0
        for folder in os.listdir('train/train_sharp/'):
            index = index + 1
            if index >= 5:
                break
            for filename in os.listdir('train/train_sharp/' + folder):
                img = cv2.imread('train/train_sharp/' + folder + '/' + filename)
                img = img.transpose((2, 0, 1)).astype(np.float32)
                self.targets.append(img)

        self.inputs = np.array(self.inputs)
        self.targets = np.array(self.targets)

        self.inputs = torch.tensor(self.inputs, dtype=torch.float32)
        self.targets = torch.tensor(self.targets, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item = self.inputs[idx]
        target = self.targets[idx]

        if self.transform:
            item = self.transform(item)

        return item, target

In [5]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

num_epochs = 1
batch_size = 4
learning_rate = 0.0001

sr_resnet_model = SRResNet(scale_factor=4, channels=3).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(sr_resnet_model.parameters(), lr=learning_rate)

train_dataset = MyDataset()

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

index = 1

print("loaded_data_set")

for epoch in range(num_epochs):
    for batch in train_loader:
        low_res_images, high_res_images = batch[0].to(device), batch[1].to(device)

        optimizer.zero_grad()

        outputs = sr_resnet_model(low_res_images)

        loss = criterion(outputs, high_res_images)

        loss.backward()

        optimizer.step()

        print(f'Epoch [{index}/{400 / batch_size}], Loss: {loss.item():.4f}')
        index = index + 1

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


loaded_data_set
Epoch [1/100.0], Loss: 14843.2715
Epoch [2/100.0], Loss: 14601.9307
Epoch [3/100.0], Loss: 16216.3975
Epoch [4/100.0], Loss: 15040.2285
Epoch [5/100.0], Loss: 13374.7100
Epoch [6/100.0], Loss: 15777.0225
Epoch [7/100.0], Loss: 15783.4619
Epoch [8/100.0], Loss: 16086.2402
Epoch [9/100.0], Loss: 15097.3984


KeyboardInterrupt: 

In [36]:
import torch
import torch.nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from torch_enhance.datasets import BSDS300, Set14, Set5
from torch_enhance.models import SRResNet
from torch_enhance import metrics

from skimage.metrics import structural_similarity as ssim

class Module(pl.LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        lr, hr = batch
        sr = self(lr)
        loss = F.mse_loss(sr, hr, reduction="mean")

        # metrics
        mae = metrics.mae(sr, hr)
        psnr = metrics.psnr(sr, hr)

        ssim_value = torch.tensor([ssim(np.concatenate(hr_i.detach().cpu().numpy().transpose(1, 2, 0)),
                                         np.concatenate(sr_i.detach().cpu().numpy().transpose(1, 2, 0)),
                                         multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False,
                                        win_size=3, channel_axis=1, 
                                        data_range=sr_i.max() - sr_i.min()) for hr_i, sr_i in zip(hr, sr)]).mean()

        # Logs
        self.log("train_loss", loss)
        self.log("train_mae", mae)
        self.log("train_psnr", psnr)
        self.log("train_ssim", ssim_value.item())

        return loss

    def validation_step(self, batch, batch_idx):
        lr, hr = batch
        sr = self(lr)
        loss = F.mse_loss(sr, hr, reduction="mean")

        # metrics
        mae = metrics.mae(sr, hr)
        psnr = metrics.psnr(sr, hr)

        # Logs
        self.log("val_loss", loss)
        self.log("val_mae", mae)
        self.log("val_psnr", psnr)

        return loss

    def test_step(self, batch, batch_idx):
        lr, hr = batch
        sr = self(lr)
        loss = F.mse_loss(sr, hr, reduction="mean")

        # metrics
        mae = metrics.mae(sr, hr)
        psnr = metrics.psnr(sr, hr)

        # Logs
        self.log("test_loss", loss)
        self.log("test_mae", mae)
        self.log("test_psnr", psnr)

        return loss

In [37]:
scale_factor = 4

# Setup dataloaders
train_dataset = BSDS300(scale_factor=scale_factor)
val_dataset = Set14(scale_factor=scale_factor)
test_dataset = Set5(scale_factor=scale_factor)

# train_dataset = MyDataset()

# train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

# Define model
channels = 3 # if train_dataset.color_space == "RGB" else 1
model = SRResNet(scale_factor, channels)
module = Module(model)

trainer = pl.Trainer(max_epochs=1, accelerator="cpu")
trainer.fit(
    module,
    train_dataloader,
    val_dataloader,
)
module.eval()
trainer.test(module, test_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | SRResNet | 1.5 M 
-----------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.198     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.